# Notebook to extract hidden-states and attention heads activations from LSTM model predictions

In [1]:
import os
import glob
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from model import LSTMExtractor
from tokenizer import tokenize
from utils import set_seed
from data import Dictionary

In [2]:
def check_folder(path):
    """Create adequate folders if necessary."""
    try:
        if not os.path.isdir(path):
            check_folder(os.path.dirname(path))
            os.mkdir(path)
    except:
        pass

Defining variables:

In [3]:
template = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/text_english_run*.txt' # path to text input
language = 'english'

In [4]:
name_template = 'weights_{}_embedding-size_{}_nhid_{}_nlayers_{}_dropout_{}_wiki_kristina_english.pt'

In [5]:
rnn_types = ['LSTM'] #, 'LSTM'
ninps = ['650'] #, '600'
nhids =  ['650'] #, '300'
nlayers = ['2'] #, '1'
dropouts =  ['02'] #, '02'
vocab_path = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training'
config_path_folder = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/'
trained_model_folder = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/models/english/'
path_to_data = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations'

In [6]:
template = '/Users/alexpsq/Code/Parietal/data/text_english_run*.txt' # path to text input
config_path_folder = '/Users/alexpsq/Code/Parietal/data/configs/'
trained_model_folder = '/Users/alexpsq/Code/Parietal'
path_to_data = '/Users/alexpsq/Code/data/stimuli-representations'
vocab_path = '/Users/alexpsq/Code/data/'

In [7]:
pretrained_lstm_models = [
    trained_model_folder + name_template.format(rnn_type, ninp, nhid, nlayer, dropout) for (rnn_type, ninp, nhid, nlayer, dropout) in zip(rnn_types, ninps, nhids, nlayers, dropouts)] # path to the model from which we want to retrieve the activations
infos = [os.path.basename(model).split('_') for model in pretrained_lstm_models]
names = ['_'.join(os.path.basename(model).split('.')[0].split('_')[1:]) for model in pretrained_lstm_models]
config_paths = [os.path.join(config_path_folder, 'config_' + name + '.yml') for name in names]
saving_path_folders = [
    os.path.join(path_to_data, '{}/{}'.format(language, name)) for name in names]
prediction_types = ['sentence' for i in pretrained_lstm_models]

In [8]:
saving_path_folders

['/Users/alexpsq/Code/data/stimuli-representations/english/LSTM_embedding-size_650_nhid_650_nlayers_2_dropout_02_wiki_kristina_english']

Creating iterator for each run:

In [9]:
paths = sorted(glob.glob(template))

In [10]:
vocab = Dictionary(vocab_path, language)

In [11]:
iterator_list = [tokenize(path, language, train=False, vocab=vocab) for path in paths]

  0%|          | 0/135 [00:00<?, ?it/s]

Tokenizing...
Preprocessing...
Preprocessed.
Tokenized.
Tokenizing...
Preprocessing...
Preprocessed.


  0%|          | 0/173 [00:00<?, ?it/s]

Tokenized.
Tokenizing...
Preprocessing...
Preprocessed.
Tokenized.
Tokenizing...
Preprocessing...
Preprocessed.


  0%|          | 0/216 [00:00<?, ?it/s]

Tokenized.
Tokenizing...
Preprocessing...
Preprocessed.
Tokenized.
Tokenizing...
Preprocessing...
Preprocessed.


  0%|          | 0/145 [00:00<?, ?it/s]

Tokenized.
Tokenizing...
Preprocessing...
Preprocessed.
Tokenized.
Tokenizing...
Preprocessing...
Preprocessed.


100%|██████████| 207/207 [00:00<00:00, 1419.57it/s]

Tokenized.
Tokenizing...
Preprocessing...
Preprocessed.
Tokenized.


## Activation extraction

In [12]:
for index, config in enumerate(config_paths):
    extractor = LSTMExtractor(config, language, names[index], prediction_types[index], output_hidden_states=True)
    print(extractor.name, ' - Extracting activations ...')
    for run_index, iterator in tqdm(enumerate(iterator_list)):
        print("############# Run {} #############".format(run_index))
        check_folder(saving_path_folders[index])
        activations  = extractor.extract_activations(iterator, language)
        
        activations.to_csv(os.path.join(saving_path_folders[index], 'activations_run{}.csv'.format(run_index + 1)), index=False)
        
        

0it [00:00, ?it/s]
  1%|          | 12/1894 [00:00<00:16, 113.59it/s]

LSTM_embedding-size_650_nhid_650_nlayers_2_dropout_05  - Extracting activations ...
############# Run 0 #############
1894



 90%|████████▉ | 1696/1894 [00:16<00:01, 125.54it/s]


100%|██████████| 1894/1894 [00:17<00:00, 105.85it/s]


1894


...

2530



 66%|██████▌   | 1674/2530 [00:16<00:08, 97.75it/s]


100%|██████████| 2530/2530 [00:25<00:00, 100.66it/s]


2530


9it [03:15, 21.74s/it]


In [16]:
saving_path_folders

['/Users/alexpsq/Code/data/stimuli-representations/english/LSTM_embedding-size_650_nhid_650_nlayers_2_dropout_02_wiki_kristina_english']